In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict

/home/yu.bryan.j/.pyenv/versions/tensorflow/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Read in the data and preprocess

It is noted in the original dataset that there are Nan, an artifact of excel.  This reflects period when the stock was not trading on the exchange.  As such, these values are converted into 0s.

In [2]:
df = pd.read_excel("../data/OMXS.xlsx", sheet_name="Raw_data", header=1).fillna(0)
df.head()

,Dates,PX_LAST,PX_LAST.1,PX_VOLUME,CUR_MKT_CAP,PX_LAST.2,PX_VOLUME.1,CUR_MKT_CAP.1,PX_LAST.3,PX_VOLUME.2,...,CUR_MKT_CAP.44,PX_LAST.46,PX_VOLUME.45,CUR_MKT_CAP.45,PX_LAST.47,PX_VOLUME.46,CUR_MKT_CAP.46,PX_LAST.48,PX_VOLUME.47,CUR_MKT_CAP.47
0,2000-01-03,1211.79,18.228,699665,4360.4765,33.074,825486,46797.4811,40.9572,6622620,...,0.0,140.0,257505,57212.4005,6269.7425,31400,21387.7006,3040.0,105635,9636.8002
1,2000-01-04,1182.03,18.039,766176,4315.0549,31.594,1901928,44656.5389,40.0156,17158495,...,0.0,130.0,655811,53125.8005,5458.3641,27849,18619.8805,2840.0,48014,9002.8002
2,2000-01-05,1131.30,17.659,465503,4224.2116,31.015,3944736,43739.1898,39.8273,4425945,...,0.0,121.5,974718,49652.1904,5071.1153,19125,17298.8755,2580.0,53323,8178.6002
3,2000-01-06,1131.30,17.659,465503,4224.2116,31.015,3944736,43739.1898,39.8273,4425945,...,0.0,121.5,974718,49652.1904,5071.1153,19125,17298.8755,2580.0,53323,8178.6002
4,2000-01-07,1152.54,17.849,852369,4269.6332,32.301,8697318,45848.9314,40.2981,19095385,...,0.0,132.0,985043,53943.1205,4768.6924,45104,16267.2334,2340.0,77637,7417.8002


## Investigate the companies that constitute the Index

The OMSX consists of 30 companies at any time.  Over a given period, companies may enter or exit the index.  As such, we have collected the index consitutents for each 6 month period from Januray 2001 to January 2020.

We consider two approaches training the model:
1. The model is trained on all companies that were apart of the index at any period within the 20 years
2. The model is trained only on companies in the index at any given point in time.


In [3]:
companies = pd.read_excel("../data/OMXS.xlsx", sheet_name="Index_members", header=1, skiprowslist=[0,1])
companies.head()

,Unnamed: 0,All,Company name,Unnamed: 3,January 2020,July 2019,January 2019,July 2018,January 2018,July 2017,...,July 2005,January 2005,July 2004,January 2004,July 2003,January 2003,July 2002,January 2002,July 2001,January 2001
0,NaN,OMX Index,OMXS30,NaN,GETIB SS Equity,GETIB SS Equity,GETIB SS Equity,GETIB SS Equity,GETIB SS Equity,GETIB SS Equity,...,NOKI SS Equity,NOKI SS Equity,NOKI SS Equity,NOKI SS Equity,NOKI SS Equity,NOKI SS Equity,WMB SS Equity,WMB SS Equity,WMB SS Equity,WMB SS Equity
1,NaN,GETIB SS Equity,Getinge AB,NaN,ATCOA SS Equity,ATCOA SS Equity,ATCOA SS Equity,ATCOA SS Equity,ATCOA SS Equity,ATCOA SS Equity,...,ATCOA SS Equity,ATCOA SS Equity,ATCOA SS Equity,ATCOA SS Equity,ATCOA SS Equity,ATCOA SS Equity,NOKI SS Equity,NOKI SS Equity,NOKI SS Equity,NOKI SS Equity
2,NaN,ATCOA SS Equity,Atlas Copco AB,NaN,VOLVB SS Equity,VOLVB SS Equity,VOLVB SS Equity,VOLVB SS Equity,VOLVB SS Equity,VOLVB SS Equity,...,VOLVB SS Equity,VOLVB SS Equity,VOLVB SS Equity,VOLVB SS Equity,VOLVB SS Equity,VOLVB SS Equity,ATCOA SS Equity,ATCOA SS Equity,ATCOA SS Equity,ATCOA SS Equity
3,NaN,VOLVB SS Equity,Volvo AB,NaN,TEL2B SS Equity,TEL2B SS Equity,TEL2B SS Equity,TEL2B SS Equity,TEL2B SS Equity,TEL2B SS Equity,...,TEL2B SS Equity,TEL2B SS Equity,TEL2B SS Equity,TEL2B SS Equity,TEL2B SS Equity,TEL2B SS Equity,VOLVB SS Equity,VOLVB SS Equity,VOLVB SS Equity,VOLVB SS Equity
4,NaN,TEL2B SS Equity,Tele2 AB,NaN,ELUXB SS Equity,ELUXB SS Equity,ELUXB SS Equity,ELUXB SS Equity,ELUXB SS Equity,LUNE SS Equity,...,STER SS Equity,STER SS Equity,STER SS Equity,STER SS Equity,STER SS Equity,STER SS Equity,TEL2B SS Equity,TEL2B SS Equity,TEL2B SS Equity,TEL2B SS Equity


In [4]:
company_names = companies['Company name'].to_list()[1:]
tickers = companies['All'].to_list()[1:]
print(f'There are {len(company_names)} companies.\n')
print(", ".join(company_names))
print()
print("Their tickers are: ")
print(", ".join(tickers))

There are 48 companies.

Getinge AB, Atlas Copco AB, Volvo AB, Tele2 AB, Electrolux AB, Telia Co AB, Hennes & Mauritz AB, Skanska AB, Alfa Laval AB, Telefonaktiebolaget LM Ericsson, SKF AB, Sandvik AB, AstraZeneca PLC, Securitas AB, Investor AB, ABB Ltd, Swedish Match AB, Skandinaviska Enskilda Banken AB, Svenska Handelsbanken AB, Svenska Cellulosa AB SCA, SSAB AB, Essity AB, Assa Abloy AB, Autoliv Inc, Hexagon AB, Swedbank AB, Atlas Copco AB, Kinnevik AB, Boliden AB, Nordea Bank Abp, Fingerprint Cards AB, Lundin Energy AB, Nokia Oyj, Modern Times Group MTG AB, Scania AB, Eniro AB, Vostok Gas Ltd, Nokia Oyj, Stora Enso Oyj, Holmen AB, Fabege AB, Skandia Forsakrings AB, Fabege AB/Old, WM-data AB, Pharmacia LLC, Europolitan Vodafone AB, LBI International AB, LB Icon AB

Their tickers are: 
GETIB SS Equity, ATCOA SS Equity, VOLVB SS Equity, TEL2B SS Equity, ELUXB SS Equity, TELIA SS Equity, HMB SS Equity, SKAB SS Equity, ALFA SS Equity, ERICB SS Equity, SKFB SS Equity, SAND SS Equity, AZN

In [5]:
# Construct a mapping of company to idx and vice versa, for translation facility
_to_id = {}
id_to_company = {}
id_to_ticker = {}
for i in range(len(company_names)):
    _to_id[company_names[i]] = i
    _to_id[tickers[i]] = i
    id_to_company[i] = company_names[i]
    id_to_ticker[i] = tickers[i]

## Extract Price, Volume, and Market Cap information



In [6]:
OMXS = df.filter(regex="PX_LAST*", axis=1).iloc[:,:1]
prices = df.filter(regex="PX_LAST*", axis=1).iloc[:,1:]
volume = df.filter(regex="PX_VOLUME*", axis=1)
market_cap = df.filter(regex="CUR_MKT_CAP", axis=1)
dates = df.iloc[:,0]
prices.head()

,PX_LAST.1,PX_LAST.2,PX_LAST.3,PX_LAST.4,PX_LAST.5,PX_LAST.6,PX_LAST.7,PX_LAST.8,PX_LAST.9,PX_LAST.10,...,PX_LAST.39,PX_LAST.40,PX_LAST.41,PX_LAST.42,PX_LAST.43,PX_LAST.44,PX_LAST.45,PX_LAST.46,PX_LAST.47,PX_LAST.48
0,18.228,33.074,40.9572,184.052,103.823,0.0,150.0,79.500,0.0,501.0417,...,148.0,155.75,9.3942,129.50,70.0685,111.4,0.0,140.0,6269.7425,3040.0
1,18.039,31.594,40.0156,168.312,102.381,0.0,144.0,76.875,0.0,499.2361,...,146.0,151.50,9.1508,116.75,66.3014,108.0,0.0,130.0,5458.3641,2840.0
2,17.659,31.015,39.8273,162.210,98.536,0.0,135.0,76.625,0.0,464.9305,...,147.0,148.00,9.0048,110.75,65.1713,106.0,0.0,121.5,5071.1153,2580.0
3,17.659,31.015,39.8273,162.210,98.536,0.0,135.0,76.625,0.0,464.9305,...,147.0,148.00,9.0048,110.75,65.1713,106.0,0.0,121.5,5071.1153,2580.0
4,17.849,32.301,40.2981,162.531,100.699,0.0,138.5,79.125,0.0,462.2222,...,157.0,157.00,9.0535,115.75,66.6781,104.6,0.0,132.0,4768.6924,2340.0


## Generate data and labels

We will investigate various network's ability to present estimates for 1, 5, and 30 days into the future.


In [7]:
forecast_periods = [1,5, 30]
data = defaultdict(lambda: defaultdict(int))
for forecast_period in forecast_periods:
    data[forecast_period]['X'] = np.asarray([prices.iloc[i].to_numpy() for i in range(prices.shape[0]-forecast_period)])
    data[forecast_period]['Y'] = np.asarray([prices.iloc[i+forecast_period].to_numpy() for i in range(prices.shape[0]-forecast_period)])


In [8]:
data[1]['X'].shape
data[1]['Y'].shape

(5299, 48)

## Dealing with companies that are no longer in the index

We sanitize our dataset to remove information from companies that do not consitute the OMSX index for any given six month period.  This way, the network will not be given information not available at a given period.

In [9]:
constituents = companies.filter(regex="January|July", axis=1).iloc[:30]
constituents.tail()

,January 2020,July 2019,January 2019,July 2018,January 2018,July 2017,January 2017,July 2016,January 2016,July 2015,...,July 2005,January 2005,July 2004,January 2004,July 2003,January 2003,July 2002,January 2002,July 2001,January 2001
25,SWEDA SS Equity,SWEDA SS Equity,SWEDA SS Equity,SWEDA SS Equity,SWEDA SS Equity,SWEDA SS Equity,SWEDA SS Equity,SWEDA SS Equity,SWEDA SS Equity,SWEDA SS Equity,...,ATCOB SS Equity,ATCOB SS Equity,ENRO SS Equity,ENRO SS Equity,ENRO SS Equity,ENRO SS Equity,ATCOB SS Equity,ATCOB SS Equity,ATCOB SS Equity,HOLMB SS Equity
26,ATCOB SS Equity,ATCOB SS Equity,ATCOB SS Equity,ATCOB SS Equity,ATCOB SS Equity,ATCOB SS Equity,ATCOB SS Equity,ATCOB SS Equity,ATCOB SS Equity,ATCOB SS Equity,...,ENRO SS Equity,ENRO SS Equity,HOLMB SS Equity,HOLMB SS Equity,HOLMB SS Equity,HOLMB SS Equity,ENRO SS Equity,ENRO SS Equity,ENRO SS Equity,SDIA SS Equity
27,KINVB SS Equity,KINVB SS Equity,KINVB SS Equity,KINVB SS Equity,KINVB SS Equity,KINVB SS Equity,KINVB SS Equity,KINVB SS Equity,KINVB SS Equity,KINVB SS Equity,...,HOLMB SS Equity,HOLMB SS Equity,SDIA SS Equity,SDIA SS Equity,SDIA SS Equity,SDIA SS Equity,HOLMB SS Equity,HOLMB SS Equity,HOLMB SS Equity,LBI SS Equity
28,BOL SS Equity,BOL SS Equity,BOL SS Equity,BOL SS Equity,BOL SS Equity,BOL SS Equity,BOL SS Equity,BOL SS Equity,BOL SS Equity,BOL SS Equity,...,SDIA SS Equity,SDIA SS Equity,FABGB SS Equity,FABGB SS Equity,FABGB SS Equity,NDA SS Equity,SDIA SS Equity,SDIA SS Equity,SDIA SS Equity,NDA SS Equity
29,NDA SS Equity,NDA SS Equity,NDA SS Equity,NDA SS Equity,NDA SS Equity,NDA SS Equity,NDA SS Equity,NDA SS Equity,NDA SS Equity,NDA SS Equity,...,NDA SS Equity,NDA SS Equity,NDA SS Equity,NDA SS Equity,NDA SS Equity,NaN,NDA SS Equity,NDA SS Equity,NDA SS Equity,ICON SS Equity


In [10]:
# Construct a dataset with only the companies at any point in time
data = prices.copy()
data.head()

,PX_LAST.1,PX_LAST.2,PX_LAST.3,PX_LAST.4,PX_LAST.5,PX_LAST.6,PX_LAST.7,PX_LAST.8,PX_LAST.9,PX_LAST.10,...,PX_LAST.39,PX_LAST.40,PX_LAST.41,PX_LAST.42,PX_LAST.43,PX_LAST.44,PX_LAST.45,PX_LAST.46,PX_LAST.47,PX_LAST.48
0,18.228,33.074,40.9572,184.052,103.823,0.0,150.0,79.500,0.0,501.0417,...,148.0,155.75,9.3942,129.50,70.0685,111.4,0.0,140.0,6269.7425,3040.0
1,18.039,31.594,40.0156,168.312,102.381,0.0,144.0,76.875,0.0,499.2361,...,146.0,151.50,9.1508,116.75,66.3014,108.0,0.0,130.0,5458.3641,2840.0
2,17.659,31.015,39.8273,162.210,98.536,0.0,135.0,76.625,0.0,464.9305,...,147.0,148.00,9.0048,110.75,65.1713,106.0,0.0,121.5,5071.1153,2580.0
3,17.659,31.015,39.8273,162.210,98.536,0.0,135.0,76.625,0.0,464.9305,...,147.0,148.00,9.0048,110.75,65.1713,106.0,0.0,121.5,5071.1153,2580.0
4,17.849,32.301,40.2981,162.531,100.699,0.0,138.5,79.125,0.0,462.2222,...,157.0,157.00,9.0535,115.75,66.6781,104.6,0.0,132.0,4768.6924,2340.0


In [11]:

for period in reversed(list(constituents.keys())):
    start_date = pd.to_datetime(period)
    end_date = start_date + pd.DateOffset(months=6)
    
    # Get the records that fall within the period
    mask = (df['Dates'] > start_date) & (df['Dates'] <= end_date)
    
    # Get the prices for those records within period
    in_period_prices = df.loc[mask].filter(regex='PX_LAST',axis=1).iloc[:,1:]
    
    # Get the companies that constitue the index for this period
    companies = constituents[period]
    companies_in_index = [_to_id[company] for company in companies if company in _to_id]
    
    # Get the companies that are not in the index for this period
    companies_not_in_index = [i for i in range(len(company_names)) if i not in companies_in_index]
    
    # Mask the prices of companies not in the index to 0
    in_period_prices.iloc[:,companies_not_in_index] = 0
    
    # Store the results
    data.loc[mask] = in_period_prices
    

In [12]:
data.head()

,PX_LAST.1,PX_LAST.2,PX_LAST.3,PX_LAST.4,PX_LAST.5,PX_LAST.6,PX_LAST.7,PX_LAST.8,PX_LAST.9,PX_LAST.10,...,PX_LAST.39,PX_LAST.40,PX_LAST.41,PX_LAST.42,PX_LAST.43,PX_LAST.44,PX_LAST.45,PX_LAST.46,PX_LAST.47,PX_LAST.48
0,18.228,33.074,40.9572,184.052,103.823,0.0,150.0,79.500,0.0,501.0417,...,148.0,155.75,9.3942,129.50,70.0685,111.4,0.0,140.0,6269.7425,3040.0
1,18.039,31.594,40.0156,168.312,102.381,0.0,144.0,76.875,0.0,499.2361,...,146.0,151.50,9.1508,116.75,66.3014,108.0,0.0,130.0,5458.3641,2840.0
2,17.659,31.015,39.8273,162.210,98.536,0.0,135.0,76.625,0.0,464.9305,...,147.0,148.00,9.0048,110.75,65.1713,106.0,0.0,121.5,5071.1153,2580.0
3,17.659,31.015,39.8273,162.210,98.536,0.0,135.0,76.625,0.0,464.9305,...,147.0,148.00,9.0048,110.75,65.1713,106.0,0.0,121.5,5071.1153,2580.0
4,17.849,32.301,40.2981,162.531,100.699,0.0,138.5,79.125,0.0,462.2222,...,157.0,157.00,9.0535,115.75,66.6781,104.6,0.0,132.0,4768.6924,2340.0


In [13]:
data.tail()

,PX_LAST.1,PX_LAST.2,PX_LAST.3,PX_LAST.4,PX_LAST.5,PX_LAST.6,PX_LAST.7,PX_LAST.8,PX_LAST.9,PX_LAST.10,...,PX_LAST.39,PX_LAST.40,PX_LAST.41,PX_LAST.42,PX_LAST.43,PX_LAST.44,PX_LAST.45,PX_LAST.46,PX_LAST.47,PX_LAST.48
5295,192.45,346.9,125.05,135.05,130.85,35.44,135.50,173.10,173.90,86.26,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5296,194.85,335.9,120.70,124.60,127.15,34.27,129.90,166.05,164.95,81.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5297,189.40,350.6,126.40,127.00,126.95,34.05,128.05,170.30,168.90,85.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5298,191.80,369.5,119.70,127.20,128.00,33.53,128.45,171.15,171.00,84.04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5299,184.95,345.0,116.20,128.50,127.00,33.86,124.75,171.90,168.50,81.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Generate the labels for each sample

In [14]:
# Define the forecast period
forecast_period = 30

# Reinitialize the OMXS dataframe
OMXS = df.filter(regex="PX_LAST*", axis=1).iloc[:,:1]

# Iterate through the frame and 
for i in range(len(OMXS)-forecast_period):
    OMXS.loc[i,('lookahead')] = OMXS.loc[i+forecast_period,('PX_LAST')]

# Set the labels for values outside of the forecast period to 0
for i in range(len(OMXS)-forecast_period, len(OMXS)):
    OMXS.loc[i,('lookahead')] = 0
    
# Determine the label based on 
OMXS['Label'] = np.where(OMXS['lookahead'] > OMXS['PX_LAST'],1, 0)


OMXS.head()

,PX_LAST,lookahead,Label
0,1211.79,1350.31,1
1,1182.03,1309.94,1
2,1131.30,1343.31,1
3,1131.30,1352.06,1
4,1152.54,1371.35,1


In [15]:
OMXS.tail()

,PX_LAST,lookahead,Label
5295,1541.166,0.0,0
5296,1493.454,0.0,0
5297,1527.633,0.0,0
5298,1541.889,0.0,0
5299,1514.130,0.0,0


## Save the data

We save the processed data into a simple pickle file for the network to train on

In [16]:
def create_lookahead_dataset(data, labels, forecast_period=1):

    # Reinitialize the OMXS lookahead_labelframe
    lookahead_label = labels.filter(regex="PX_LAST*", axis=1).iloc[:,:1]

    # Iterate through the frame and 
    for i in range(len(lookahead_label)-forecast_period):
        lookahead_label.loc[i,('lookahead')] = lookahead_label.loc[i+forecast_period,('PX_LAST')]

    # Set the labels for values outside of the forecast period to 0
    for i in range(len(lookahead_label)-forecast_period, len(lookahead_label)):
        lookahead_label.loc[i,('lookahead')] = 0

    # Determine the label based on 
    lookahead_label['Label'] = np.where(lookahead_label['lookahead'] > lookahead_label['PX_LAST'],1, 0)
    
    lookahead_data = data.iloc[:-forecast_period,:].to_numpy()
    lookahead_labels = lookahead_label.iloc[:-forecast_period,[2]].to_numpy()

    
    return lookahead_data, lookahead_labels

In [17]:
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit
sss = ShuffleSplit(n_splits=1, test_size=0.2)

for forecast_period in [5,10,20]:
    X, y = create_lookahead_dataset(data, df, forecast_period=forecast_period)
    
    sss.get_n_splits(X, y)
    for train_index, test_index in sss.split(X, y):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        np.save(f'../data/processed/OMXS_processed_forecast{forecast_period}_train.npy',X_train)
        np.save(f'../data/processed/OMXS_labels_forecast{forecast_period}_train.npy',y_train)
        np.save(f'../data/processed/OMXS_processed_forecast{forecast_period}_test.npy',X_test)
        np.save(f'../data/processed/OMXS_labels_forecast{forecast_period}_test.npy',y_test)

TRAIN: [2249 4475 2193 ... 2855 1027 4166] TEST: [3483 4419 2983 ... 1286 3040 3615]
TRAIN: [4540  391 4689 ... 3294 5113 1163] TEST: [4133 2007 5097 ... 3259  928 4623]
TRAIN: [3796  514  701 ... 1790 1058 4122] TEST: [2943 2095 4980 ... 2639  135 2506]
